In [1]:
import random
from collections import Counter

from presidio_evaluator.data_generator import read_synth_dataset
from presidio_evaluator import InputSample
from presidio_evaluator.models.crf_model import CRFModel
from presidio_evaluator.validation import split_dataset, save_to_json

import sklearn_crfsuite
from sklearn_crfsuite import metrics

/home/fatemeh.rahimi/.virtualenvs/address/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


Flair is not installed by default
Flair is not installed


In [2]:
DATA_PATH = "../../presidio-research/data/generated_address_size_1000_date_November_07_2021.json"

# Baslines
* CRF (conditional Random Field)
* spacy (Evaluate with existing Spacy model)

## Data Split
Note that we don't want the same pattern to be in more than one set. ([code sample](https://github.com/microsoft/presidio-research/blob/master/notebooks/Split%20by%20pattern%20%23.ipynb))    
note that `split_dataset` function is based on `template#` in `meta_data` 

In [3]:
all_samples = read_synth_dataset(DATA_PATH)
print(len(all_samples))


1000


In [4]:
all_samples[0]

Full text: X-Folder: \Vincent_Kaminski_Jun2001_9\Notes Folders\Audit
X-Origin: Kaminski-V
X-FileName: vkamins.nsf

Professor Darrel Duffie
24201 North Highway One

         May 8, 2000
Spans: [Type: LOCATION, value: 24201 North Highway One, start: 128, end: 151]
Tokens: [X, -, Folder, :, \Vincent_Kaminski_Jun2001_9\Notes, Folders\Audit, 
, X, -, Origin, :, Kaminski, -, V, 
, X, -, FileName, :, vkamins.nsf, 

, Professor, Darrel, Duffie, 
, 24201, North, Highway, One, 

         , May, 8, ,, 2000]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'L-LOCATION', 'O', 'O', 'O', 'O', 'O']

In [5]:
TRAIN_TEST_RATIOS = [0.7,0.3]
train_data,test_data = split_dataset(all_samples, TRAIN_TEST_RATIOS)

In [6]:
train_data[0]

Full text: Subject: Margie Harris


Margie Harris

319 West 48th Street
uturn 1400 Smith Street, EB4771 
Houston, Tx 77001713) 853-7172 on right, lavander house
Spans: [Type: LOCATION, value: 319 West 48th Street, start: 40, end: 60, Type: LOCATION, value: 1400 Smith Street, EB4771 
Houston, Tx 77001713) 853-7172, start: 67, end: 124]
Tokens: [Subject, :, Margie, Harris, 


, Margie, Harris, 

, 319, West, 48th, Street, 
, uturn, 1400, Smith, Street, ,, EB4771, 
, Houston, ,, Tx, 77001713, ), 853, -, 7172, on, right, ,, lavander, house]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'L-LOCATION', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'L-LOCATION', 'O', 'O', 'O', 'O', 'O']

## CRF

In [7]:
# all_samples = read_synth_dataset(DATA_PATH)
# all_samples = [sample for sample in all_samples if len(sample.spans) > 0]
# print("Kept {} samples after removal of non-tagged samples".format(len(all_samples)))

# random.shuffle(all_samples)

# train_len = int(len(all_samples)* 0.80)
# train_data = all_samples[:train_len]
# test_data = all_samples[train_len:]

train_data_CRF = InputSample.create_conll_dataset(train_data, to_bio=False)
test_data_CRF = InputSample.create_conll_dataset(test_data, to_bio=False)

test_data_CRF.head()

,text,pos,tag,label,sentence
0,Wednesdays,NOUN,NNS,O,0
1,and,CCONJ,CC,O,0
2,wo,AUX,MD,O,0
3,n't,PART,RB,O,0
4,drive,VERB,VB,O,0


In [8]:
# Turn every sentence into a list of lists (list of tokens + pos + label)
test_sents=test_data_CRF.groupby('sentence')[['text','pos','label']].apply(lambda x: x.values.tolist())
train_sents=train_data_CRF.groupby('sentence')[['text','pos','label']].apply(lambda x: x.values.tolist())

In [9]:
CRFModel.sent2features(train_sents[0])[0]

{'bias': 1.0,
 'word.lower()': 'subject',
 'word[-3:]': 'ect',
 'word[-2:]': 'ct',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'postag': 'NOUN',
 'postag[:2]': 'NO',
 'BOS': True,
 '+1:word.lower()': ':',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'PUNCT',
 '+1:postag[:2]': 'PU'}

In [10]:
%time
X_train = [CRFModel.sent2features(s) for s in train_sents]
y_train = [CRFModel.sent2labels(s) for s in train_sents]

X_test = [CRFModel.sent2features(s) for s in test_sents]
y_test = [CRFModel.sent2labels(s) for s in test_sents]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [11]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 3.63 s, sys: 3.69 ms, total: 3.63 s
Wall time: 3.63 s


/home/fatemeh.rahimi/.virtualenvs/address/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [12]:
import pickle
with open("../model_weights/crf.pickle",'wb') as f:
    pickle.dump(crf, f,protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
import pickle
with open("../model_weights/crf.pickle", 'rb') as f:
    crf = pickle.load(f)

In [14]:
labels = list(crf.classes_)
# labels.remove('O')
labels

['O', 'B-GPE', 'I-GPE', 'L-GPE', 'U-GPE']

In [15]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9861090774209577

In [16]:
y_5_pred = crf.predict([X_test[5]])
# y_5_pred[0]

In [17]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           O      0.994     0.991     0.992     20714
       B-GPE      0.941     0.939     0.940       358
       I-GPE      0.948     0.968     0.958      2924
       L-GPE      0.908     0.905     0.906       358
       U-GPE      1.000     0.500     0.667         4

    accuracy                          0.986     24358
   macro avg      0.958     0.861     0.893     24358
weighted avg      0.986     0.986     0.986     24358



/home/fatemeh.rahimi/.virtualenvs/address/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['O', 'B-GPE', 'I-GPE', 'L-GPE', 'U-GPE'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


## predict on test set without newline

In [18]:
test_data[0].full_text.replace("\n", " ")

"Wednesdays and won't drive back on I-40 if it's after 3:00, I take a back route.  Let me give you directions to Orton just in case, it's easy.  I-40 from Raleigh to Wilmington, as you approach Wilmington take a right at a big light on 600 Fifth Avenue Rockefeller Center - 27th floor New York, NY 10020 (new since your day, it's just before 10435 N Tantau Ave, Cupertino  Old-Bolton Rd.), go about 1-2 miles and it will end, take a right on 5959 Topanga Canyon Blvd.  Suite 244 Woodland Hills, CA 91367, pass the airport on your right, road will come to an end again, take a left, I think this is Hwy 133 South which takes you around/through nig-town Wilmington, cross the Cape Fear River and follow signs to Southport staying"

In [19]:
#remove new line
test_data_no_newline = []
for data_with_newline in test_data:
    data_with_newline.full_text = data_with_newline.full_text.replace("\n", " ")
    test_data_no_newline.append(data_with_newline)

#test
test_data_CRF = InputSample.create_conll_dataset(test_data_no_newline, to_bio=False)
test_sents=test_data_CRF.groupby('sentence')[['text','pos','label']].apply(lambda x: x.values.tolist())

X_test = [CRFModel.sent2features(s) for s in test_sents]
y_test = [CRFModel.sent2labels(s) for s in test_sents]

import pickle
with open("../model_weights/crf.pickle", 'rb') as f:
    crf = pickle.load(f)

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           O      0.994     0.991     0.992     20714
       B-GPE      0.941     0.939     0.940       358
       I-GPE      0.948     0.968     0.958      2924
       L-GPE      0.908     0.905     0.906       358
       U-GPE      1.000     0.500     0.667         4

    accuracy                          0.986     24358
   macro avg      0.958     0.861     0.893     24358
weighted avg      0.986     0.986     0.986     24358



## Evaluate an existing spaCy trained model
* Using [this noteboook](https://github.com/microsoft/presidio-research/blob/master/notebooks/models/Evaluate%20spacy%20models.ipynb)

In [20]:
import spacy

from collections import Counter

from presidio_evaluator.models import SpacyModel

from presidio_evaluator.evaluation import Evaluator, ModelError
from presidio_evaluator.data_generator import read_synth_dataset
%reload_ext autoreload
%autoreload 2

synth_samples = read_synth_dataset(DATA_PATH)
print(len(synth_samples))
DATASET = synth_samples

1000


In [21]:
entity_counter = Counter()
for sample in DATASET:
    for span in sample.spans:
        entity_counter[span.entity_type]+=1

In [22]:
entity_counter

Counter({'LOCATION': 1250})

In [23]:
DATASET[1]

Full text: * Asia Pacific Energy - April 18 

Location and Accommodations: 

The Four Seasons Hotel 
19191 Vallco Pkwy , Cupertino  200 Forest Street
Tel.: +1 713 650 1300 
Fax: +1 713 650 1203 
* Please contact the hotel directly for room reservations.
Spans: [Type: LOCATION, value: 19191 Vallco Pkwy , Cupertino  200 Forest Street, start: 90, end: 138]
Tokens: [*, Asia, Pacific, Energy, -, April, 18, 

, Location, and, Accommodations, :, 

, The, Four, Seasons, Hotel, 
, 19191, Vallco, Pkwy, ,, Cupertino,  , 200, Forest, Street, 
, Tel, ., :, +1, 713, 650, 1300, 
, Fax, :, +1, 713, 650, 1203, 
, *, Please, contact, the, hotel, directly, for, room, reservations, .]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'I-LOCATION', 'L-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [24]:
#max length sentence
max([len(sample.tokens) for sample in DATASET])

332

In [25]:
models = [
    "en_core_web_lg", 
    "en_core_web_trf",
    ]

In [26]:
for model in models:
    print("-----------------------------------")
    print("Evaluating model {}".format(model))
    nlp = spacy.load(model)
    spacy_model = SpacyModel(model=nlp,entities_to_keep=['GPE'])
    evaluator = Evaluator(model=spacy_model)
    evaluation_results = evaluator.evaluate_all(DATASET)
    scores = evaluator.calculate_score(evaluation_results)
    
    print("Confusion matrix:")
    print(scores.results)

    print("Precision and recall")
    scores.print()
    errors = scores.model_errors

-----------------------------------
Evaluating model en_core_web_lg


Evaluating <class 'presidio_evaluator.evaluation.evaluator.Evaluator'>:   1%|          | 10/1000 [00:00<00:10, 96.54it/s]

Translating entites using this dictionary: {'ORGANIZATION': 'ORG', 'COUNTRY': 'GPE', 'CITY': 'GPE', 'LOCATION': 'GPE', 'PERSON': 'PERSON', 'FIRST_NAME': 'PERSON', 'LAST_NAME': 'PERSON', 'NATION_MAN': 'GPE', 'NATION_WOMAN': 'GPE', 'NATION_PLURAL': 'GPE', 'NATIONALITY': 'GPE', 'GPE': 'GPE', 'ORG': 'ORG'}


Evaluating <class 'presidio_evaluator.evaluation.evaluator.Evaluator'>: 100%|██████████| 1000/1000 [00:12<00:00, 79.95it/s]


Confusion matrix:
Counter({('O', 'O'): 61596, ('GPE', 'O'): 10536, ('GPE', 'GPE'): 1536, ('O', 'GPE'): 468})
Precision and recall
                        Entity                     Precision                        Recall
                           GPE                        76.65%                        12.72%
                           PII                        76.65%                        12.72%
PII F measure: 0.14377566039197456
-----------------------------------
Evaluating model en_core_web_trf


Evaluating <class 'presidio_evaluator.evaluation.evaluator.Evaluator'>:   0%|          | 0/1000 [00:00<?, ?it/s]

Translating entites using this dictionary: {'ORGANIZATION': 'ORG', 'COUNTRY': 'GPE', 'CITY': 'GPE', 'LOCATION': 'GPE', 'PERSON': 'PERSON', 'FIRST_NAME': 'PERSON', 'LAST_NAME': 'PERSON', 'NATION_MAN': 'GPE', 'NATION_WOMAN': 'GPE', 'NATION_PLURAL': 'GPE', 'NATIONALITY': 'GPE', 'GPE': 'GPE', 'ORG': 'ORG'}


/home/fatemeh.rahimi/.virtualenvs/address/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Evaluating <class 'presidio_evaluator.evaluation.evaluator.Evaluator'>: 100%|██████████| 1000/1000 [02:07<00:00,  7.83it/s]

Confusion matrix:
Counter({('O', 'O'): 61584, ('GPE', 'O'): 10365, ('GPE', 'GPE'): 1707, ('O', 'GPE'): 480})
Precision and recall
                        Entity                     Precision                        Recall
                           GPE                        78.05%                        14.14%
                           PII                        78.05%                        14.14%
PII F measure: 0.15940530932416241


## prepare training data for spacy

In [27]:
# train_data
train_tagged = [sample for sample in train_data if len(sample.spans)>0]
print("Kept {} samples after removal of non-tagged samples(for train)".format(len(train_tagged)))

test_tagged = [sample for sample in test_data if len(sample.spans)>0]
print("Kept {} samples after removal of non-tagged samples(for test)".format(len(test_tagged)))




print("Entities found in training set:")
entities = []
for sample in train_tagged:
    entities.extend([tag for tag in sample.tags])
set(entities)

Kept 683 samples after removal of non-tagged samples(for train)
Kept 317 samples after removal of non-tagged samples(for test)
Entities found in training set:


{'B-LOCATION', 'I-LOCATION', 'L-LOCATION', 'O', 'U-LOCATION'}

In [28]:
from spacy.util import filter_spans
from spacy.tokens import DocBin
import os

SPACY_ROOT_PATH = "../data/spacy/"

if not os.path.exists(SPACY_ROOT_PATH):
    os.mkdir(SPACY_ROOT_PATH)

def convert_df_to_spacy(samples, file_name):
    nlp = spacy.blank("en")

    # the DocBin will store the example documents
    db = DocBin()
    for sample in samples:
        
        text = sample.full_text
        # text = text.replace("\n", " ")
        doc = nlp(text)
        ents = []
        annotations =[ (item.start_position, item.end_position) for item in sample.spans]
        # print(i)
        # print(annotations)
        for start, end in annotations:
            
            span = doc.char_span(start, end, label="GPE", alignment_mode="expand")
            # print(span)
            ents.append(span)
        
        #to solve overlaping entites (drops them)
        ents = filter_spans(ents)
        
        doc.ents = ents
        db.add(doc)
    db.to_disk(f"{SPACY_ROOT_PATH}/{file_name}")
    print(f"{file_name} successfully saved!")

convert_df_to_spacy(train_tagged,"train.spacy")
convert_df_to_spacy(test_tagged,"dev.spacy")
# convert_df_to_spacy(df_test,"test.spacy")

train.spacy successfully saved!
dev.spacy successfully saved!


training a spacy model:

1. `python -m spacy init fill-config base_config.cfg config.cfg`
2.  `python -m spacy train config/config.cfg --output ./output --paths.train ./data/spacy/train.spacy --paths.dev ./data/spacy/dev.spacy`
3. `python -m spacy evaluate "./output/model-best" "./data/spacy/dev.spacy"`

In [1]:
! python -m spacy train ../config/config.cfg --output ../output --paths.train ../data/spacy/train.spacy --paths.dev ../data/spacy/dev.spacy

/home/fatemeh.rahimi/.virtualenvs/address/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-11-29 11:48:52,246] [INFO] Set up nlp object from config
[2021-11-29 11:48:52,253] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-11-29 11:48:52,256] [INFO] Created vocabulary
[2021-11-29 11:49:05,369] [INFO] Added vectors: en_core_web_lg
[2021-11-29 11:49:05,369] [INFO] Finished initializing nlp object
[2021-11-29 11:49:07,948] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline ======================

In [4]:
! python -m spacy evaluate "../output/model-best" "../data/spacy/dev.spacy"

/home/fatemeh.rahimi/.virtualenvs/address/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   75.85 
NER R   73.76 
NER F   74.79 
SPEED   4958  


=============================== NER (per type) ===============================

          P       R       F
GPE   75.85   73.76   74.79

